# Insula API Data Collections

## Overview

This document serves as a quick reference guide of how to access Insula's APIs, using Python. This guide will show how to create a collection and upload data to it.


***Steps***:
1. Preparation
2. Create a Collection
3. Upload Data to a Collection

## Preparation

These preparation blocks have to be executed as a prerequisite for every subsequent step to work correctly as the imports will download every necessary package.
Insula uses the OIDC authentication, so check that the endpoints are correct and be sure to put a valid `USERNAME/TOKEN` pair.

In [ ]:
import requests
import base64
import json
import getpass

In [ ]:
USERNAME=getpass.getpass()

In [ ]:
PASSWORD=getpass.getpass()

To authenticate wWith `InsulaWorkflowClient` one must configure every endpoint correctly and is more generic.
The `InsulaOpenIDConnect` function will take care of providing or refreshing the token for the authentication every time `get_authorization_header()` is invoked.
This snippet can be run again if for any reason the token is expired.

In [ ]:
from InsulaWorkflowClient import InsulaOpenIDConnect

BASE_URL="https://biomass.pal.maap.eo.esa.int"

insulaAuth: InsulaOpenIDConnect = InsulaOpenIDConnect(
        authorization_endpoint="https://identity.pal.maap.eo.esa.int/realms/biomass/protocol/openid-connect/auth",
        token_endpoint="https://identity.pal.maap.eo.esa.int/realms/biomass/protocol/openid-connect/token",
        redirect_uri="http://localhost:9207/auth",
        client_id="api-client"
    )
insulaAuth.set_user_credentials(username=USERNAME, password=PASSWORD)

bearer = insulaAuth.get_authorization_header()
print(bearer)
HEADERS={'Authorization': bearer }

## Create a Collection

For creating a new collection we need to pass a JSON object with its details along with the POST HTTP request.
Inside `coll` you can see how the JSON object is made, feel free to modify its values.

> Be mindful when running this one as if a collection with the same name is already present in the platform the command will fail with HTTP 409, so try with a different collection or delete the old one to test it a second time.

In [ ]:
coll={'name': "JupyterDemoNewColl", 'description': "no", 'fileType': "REFERENCE_DATA", 'productsType': "no"}

url=BASE_URL + "/secure/api/v2.0/collections/"
run_request=requests.post(url, headers=HEADERS, json=coll)
run_request_dict = json.loads(run_request.text)

if run_request.status_code>=400: print(str(run_request.status_code)+" ERROR: Could not create Collection")
else: print(str(run_request.status_code) +" Collection successfully created")

In [ ]:
run_request_dict

## Upload Data to a Collection

For this step a file (`filename`) is created form scratch and uploaded, feel free to modify it with an already existing one and if you do remove the `truncate()` line.

The object (`uploads`) has to be created with the structure below and passed under the `files` field of the http request to work correctly.

We're using the `refData` API to upload also specifying the collection and the file type.

In [ ]:
file_type='OTHER'
collection_identifier="biomass4ac20d9a375f4200b4b982d7bcc96a90" #JupyterDemo collection
filename="uploadTestFile.tif"
metadata={} #put optional metadata fields here

with open(filename, "w+b") as upload:
    upload.truncate(1024) #just to produce a 1kb file
    uploads={
        'file':(filename, upload, 'image/tif'),
        'userProperties': (None, json.dumps(metadata), 'application/json')
    }
    
    url=BASE_URL + f"/secure/api/v2.0/platformFiles/refData?fileType={file_type}&collection={collection_identifier}"
    run_request=requests.post(url, headers=HEADERS, files=uploads)
    run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict